# MODEL TRAINING - SET 3

### 1.1 Import Data and Required Packages
#### Importing Pandas, Numpy, Matplotlib, Seaborn and Warnings Library.

In [366]:
#Importing recquired packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

##### Importing Modelling Libraries

In [367]:
#Importing ML Packages
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.multioutput import RegressorChain

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.cross_decomposition import PLSRegression 
from sklearn.svm import SVR                          
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor   

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor

from xgboost import XGBRegressor

#Importing DL Packages
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, TensorBoard
from keras.optimizers import Adam
import datetime

#### Importing CSV data as Pandas dataframe

In [368]:
#Importing CSV data as Pandas dataframe
df=pd.read_csv('dataset/Extracted_CrudeData.csv')

#Show top 5 rows of the dataframe
df.head()

,Crude Name,StdLiquidDensity (kg/m3),SulfurByWt (%),ConradsonCarbonByWt (%),NitrogenByWt (%),Distillation Mass @ X Pct (C)@ 1 (%) - TBP,Distillation Mass @ X Pct (C)@ 5 (%) - TBP,Distillation Mass @ X Pct (C)@ 10 (%) - TBP,Distillation Mass @ X Pct (C)@ 30 (%) - TBP,Distillation Mass @ X Pct (C)@ 50 (%) - TBP,...,KinematicViscosity (cSt)@ 37.78 (C),KinematicViscosity (cSt)@ 98.89 (C),CetaneNumber,BromineNumber,AnilinePoint (C),FreezePoint (C),PourPoint (C),CloudPoint (C),TotalAcidNumber (mg KOH/g),CtoHRatioByWt
0,Achinsk-2015,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,...,57.2992918615153,7.64442822441154,50.267061,NaN,65.639336,108.476408,31.114131,35.355448,0.121042,7.220029
1,Akpo-2014,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,...,1.7573028543796,0.833437569080932,48.034358,NaN,10.907278,-32.769863,13.927943,20.861818,0.080432,5.829529
2,AlJurf-2014,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,...,7.05914911333451,2.02549206169594,53.504942,NaN,40.198516,5.823258,25.601385,31.622843,0.055671,6.654117
3,AlJurf-2015,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,...,6.77137102006056,2.01032962388146,54.213282,NaN,39.958106,18.377083,27.135976,32.684943,0.028494,6.658939
4,Alba-1994,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,...,208.950655083803,7.7853062269412,57.443852,NaN,76.496875,1.073170,13.930097,40.746252,1.127930,7.499064


In [369]:
#Shape of dataset
df.shape

(114, 27)

In [370]:
#Renaming columns for better readability
df.rename(columns={
    'Crude Name': 'Crude_Name',
    'StdLiquidDensity (kg/m3)': 'StdLiqDensity_kgm3',
    'SulfurByWt (%)': 'SulfurWt_pct',
    'ConradsonCarbonByWt (%)': 'ConradsonCarbon_wt_pct',
    'NitrogenByWt (%)': 'NitrogenWt_pct',
    
    'Distillation Mass @ X Pct (C)@ 1 (%) - TBP': 'TBP_TempAt_1pct',
    'Distillation Mass @ X Pct (C)@ 5 (%) - TBP': 'TBP_TempAt_5pct',
    'Distillation Mass @ X Pct (C)@ 10 (%) - TBP': 'TBP_TempAt_10pct',
    'Distillation Mass @ X Pct (C)@ 30 (%) - TBP': 'TBP_TempAt_30pct',
    'Distillation Mass @ X Pct (C)@ 50 (%) - TBP': 'TBP_TempAt_50pct',
    'Distillation Mass @ X Pct (C)@ 70 (%) - TBP': 'TBP_TempAt_70pct',
    'Distillation Mass @ X Pct (C)@ 90 (%) - TBP': 'TBP_TempAt_90pct',
    'Distillation Mass @ X Pct (C)@ 95 (%) - TBP': 'TBP_TempAt_95pct',
    'Distillation Mass @ X Pct (C)@ 99 (%) - TBP': 'TBP_TempAt_99pct',

    'AromByWt (%)': 'AromWt_pct',
    'NaphthenesByWt (%)': 'NaphWt_pct',
    'ParaffinsByWt (%)': 'ParaWt_pct',

    'KinematicViscosity (cSt)@ 37.78 (C)': 'KV_37.78c_cSt',
    'KinematicViscosity (cSt)@ 98.89 (C)': 'KV_98.89c_cSt',
    
    "CetaneNumber": "CetaneNumber",
    "BromineNumber": "BromineNumber",
    "AnilinePoint (C)": "AnilinePoint",
    "FreezePoint (C)": "FreezePoint",
    "PourPoint (C)": "PourPoint",
    "CloudPoint (C)": "CloudPoint",
    "TotalAcidNumber (mg KOH/g)": "TotalAcidNumber",
    "CtoHRatioByWt": "CtoHRatioByWt"
}, inplace=True)
df.head()


,Crude_Name,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,...,KV_37.78c_cSt,KV_98.89c_cSt,CetaneNumber,BromineNumber,AnilinePoint,FreezePoint,PourPoint,CloudPoint,TotalAcidNumber,CtoHRatioByWt
0,Achinsk-2015,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,...,57.2992918615153,7.64442822441154,50.267061,NaN,65.639336,108.476408,31.114131,35.355448,0.121042,7.220029
1,Akpo-2014,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,...,1.7573028543796,0.833437569080932,48.034358,NaN,10.907278,-32.769863,13.927943,20.861818,0.080432,5.829529
2,AlJurf-2014,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,...,7.05914911333451,2.02549206169594,53.504942,NaN,40.198516,5.823258,25.601385,31.622843,0.055671,6.654117
3,AlJurf-2015,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,...,6.77137102006056,2.01032962388146,54.213282,NaN,39.958106,18.377083,27.135976,32.684943,0.028494,6.658939
4,Alba-1994,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,...,208.950655083803,7.7853062269412,57.443852,NaN,76.496875,1.073170,13.930097,40.746252,1.127930,7.499064


#### Duplicate Values, Missing Values & Data Types

In [371]:
#Exploring data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Crude_Name              114 non-null    object 
 1   StdLiqDensity_kgm3      114 non-null    float64
 2   SulfurWt_pct            114 non-null    float64
 3   ConradsonCarbon_wt_pct  114 non-null    float64
 4   NitrogenWt_pct          114 non-null    float64
 5   TBP_TempAt_1pct         114 non-null    float64
 6   TBP_TempAt_5pct         114 non-null    float64
 7   TBP_TempAt_10pct        114 non-null    float64
 8   TBP_TempAt_30pct        114 non-null    float64
 9   TBP_TempAt_50pct        114 non-null    float64
 10  TBP_TempAt_70pct        114 non-null    float64
 11  TBP_TempAt_90pct        114 non-null    float64
 12  TBP_TempAt_95pct        114 non-null    float64
 13  TBP_TempAt_99pct        114 non-null    float64
 14  AromWt_pct              114 non-null    fl

In [372]:
#Changing data types
df["KV_37.78c_cSt"] = pd.to_numeric(df["KV_37.78c_cSt"], errors="coerce")
df["KV_98.89c_cSt"] = pd.to_numeric(df["KV_98.89c_cSt"], errors="coerce")

In [373]:
#Checks for duplicate values
df.duplicated().sum()

np.int64(0)

In [374]:
#Checks for missing values
df.isna().sum()

Crude_Name                  0
StdLiqDensity_kgm3          0
SulfurWt_pct                0
ConradsonCarbon_wt_pct      0
NitrogenWt_pct              0
TBP_TempAt_1pct             0
TBP_TempAt_5pct             0
TBP_TempAt_10pct            0
TBP_TempAt_30pct            0
TBP_TempAt_50pct            0
TBP_TempAt_70pct            0
TBP_TempAt_90pct            0
TBP_TempAt_95pct            0
TBP_TempAt_99pct            0
AromWt_pct                  0
NaphWt_pct                  0
ParaWt_pct                  0
KV_37.78c_cSt               5
KV_98.89c_cSt               2
CetaneNumber                0
BromineNumber             114
AnilinePoint                0
FreezePoint                 0
PourPoint                   0
CloudPoint                  0
TotalAcidNumber             0
CtoHRatioByWt               0
dtype: int64

In [375]:
#Deleting columns with more than 50% missing values
df.drop(columns='BromineNumber', inplace=True)
df.shape

(114, 26)

#### ML Model Training

In [376]:
#Independent Features
X = df.drop(columns=['Crude_Name', 'AromWt_pct', 'NaphWt_pct', 'ParaWt_pct', 'KV_37.78c_cSt', 'KV_98.89c_cSt',
                     'CetaneNumber', 'AnilinePoint', 'FreezePoint', 'PourPoint', 'CloudPoint',
                     'TotalAcidNumber', 'CtoHRatioByWt'])
X.head()

,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,TBP_TempAt_70pct,TBP_TempAt_90pct,TBP_TempAt_95pct,TBP_TempAt_99pct
0,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,561.500466,665.837149,714.976170,806.217469
1,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,311.635864,465.560748,539.770951,687.654783
2,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,468.885623,655.500149,748.288326,914.614065
3,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,481.574377,655.699458,736.138972,885.381839
4,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,595.448493,779.746511,866.765919,1005.918590


In [377]:
#Dependent Features
y = df[['CetaneNumber', 'AnilinePoint', 'FreezePoint', 'PourPoint', 'CloudPoint', 'TotalAcidNumber', 'CtoHRatioByWt']]

In [378]:
#Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Train size: 91, Test size: 23


((91, 13), (23, 13), (91, 7), (23, 7))

In [379]:
#Define categorical and numerical feature columns
num_features = [col for col in X.columns]

#Define transformations for each feature
numeric_transformer = StandardScaler()                    

#Apply scaling to numeric features
preprocessor = ColumnTransformer([("StandardScaler", numeric_transformer, num_features)])

In [380]:
#Machine Learning Algorithms
models = {
    "Linear Regression": Pipeline([
        ('pre', preprocessor),
        ('linreg', MultiOutputRegressor(LinearRegression()))
    ]),
    "Ridge Regression": Pipeline([
        ('pre', preprocessor),
        ('ridge', MultiOutputRegressor(Ridge(alpha=1.0)))
    ]),
    "Lasso Regression": Pipeline([
        ('pre', preprocessor),
        ('ridge', MultiOutputRegressor(Lasso(alpha=1.0)))
    ]),
    "Elastic Net Regression": Pipeline([
        ('pre', preprocessor),
        ('elasticnet', MultiOutputRegressor(ElasticNet(alpha=0.5, l1_ratio=0.5, max_iter=2000)))
    ]),
    "Support Vector Regression": Pipeline([
        ('pre', preprocessor),
        ('svr', MultiOutputRegressor(SVR(kernel='rbf', C=100, epsilon=0.1)))
    ]),
    "K-Nearest Neighbours": Pipeline([
        ('pre', preprocessor), 
        ('knn', MultiOutputRegressor(KNeighborsRegressor(n_neighbors=5)))
    ]),
    "PLS Regression": Pipeline([
        ('pre', preprocessor),
        ('pls', PLSRegression(n_components=3))
    ]),
    "Decision Tree Regression": Pipeline([
        ('pre', preprocessor),  
        ('dt', MultiOutputRegressor(DecisionTreeRegressor(max_depth=10, min_samples_split=5, random_state=42)))
    ]),
    "Extra Tree Regression": Pipeline([
        ('pre', preprocessor),  
        ('et', MultiOutputRegressor(ExtraTreesRegressor(n_estimators=200, max_depth=15, min_samples_split=5, random_state=42)))
    ]),
    "Gradient Boost": Pipeline([
        ('scaler', StandardScaler()),
        ('gb', MultiOutputRegressor(GradientBoostingRegressor(random_state=42)))
    ]),
    "AdaBoost": Pipeline([
        ('scaler', StandardScaler()),
        ('gb', MultiOutputRegressor(AdaBoostRegressor(n_estimators=100, learning_rate=1.0, random_state=42)))
    ]),
    "Random Forest": Pipeline([
        ('pre', preprocessor),
        ('rf', MultiOutputRegressor(RandomForestRegressor(n_estimators=200, random_state=42)))
    ]),
    "XGBoost": Pipeline([
        ('pre', preprocessor),
        ('xgb', MultiOutputRegressor(XGBRegressor(n_estimators=500, learning_rate=0.01, max_depth=2, reg_alpha=0.5, subsample=0.7, random_state=42)))
    ]) 
}

In [381]:
#Train and evaluate all models
print("Training ML Models...")

results = {}
for name, model in models.items():

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'Test R2': r2_score(y_test, y_pred),
        'Test MAE': mean_absolute_error(y_test, y_pred),
        'Test RMSE': np.sqrt(mean_squared_error(y_test, y_pred))
    }
results_df = pd.DataFrame(results).T
print(results_df.sort_values('Test R2', ascending=False))


Training ML Models...
                            Test R2   Test MAE  Test RMSE
AdaBoost                   0.310767  10.523865  19.341133
XGBoost                    0.268444  11.014826  20.314714
Extra Tree Regression      0.257772  10.921554  20.770111
Elastic Net Regression     0.246291  10.748352  20.434483
Random Forest              0.222258  10.588456  20.049300
K-Nearest Neighbours       0.182313  11.560373  21.270321
Gradient Boost             0.169938  11.238276  21.683881
Lasso Regression           0.157788  10.987203  20.581869
Support Vector Regression  0.121437   9.741702  19.160556
PLS Regression             0.026896  10.864044  20.448904
Ridge Regression           0.010889  11.625942  21.490482
Linear Regression         -0.223829  12.703869  22.780102
Decision Tree Regression  -0.229439  12.824256  23.356935


In [382]:
#Independent Features
X = df.drop(columns=['Crude_Name', 'AromWt_pct', 'NaphWt_pct', 'ParaWt_pct', 'KV_37.78c_cSt', 'KV_98.89c_cSt',
                     'CetaneNumber', 'AnilinePoint', 'FreezePoint', 'PourPoint', 'CloudPoint',
                     'TotalAcidNumber', 'CtoHRatioByWt'])
X.head()

,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,TBP_TempAt_70pct,TBP_TempAt_90pct,TBP_TempAt_95pct,TBP_TempAt_99pct
0,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,561.500466,665.837149,714.976170,806.217469
1,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,311.635864,465.560748,539.770951,687.654783
2,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,468.885623,655.500149,748.288326,914.614065
3,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,481.574377,655.699458,736.138972,885.381839
4,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,595.448493,779.746511,866.765919,1005.918590


In [383]:
#Dependent Features
y = df[['CetaneNumber', 'AnilinePoint', 'FreezePoint', 'PourPoint', 'CloudPoint', 'TotalAcidNumber', 'CtoHRatioByWt']]

In [384]:
#Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((91, 13), (23, 13), (91, 7), (23, 7))

In [385]:
scaler_X = StandardScaler()
scaler_y = StandardScaler() 

#Fit Train & transform Test
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled  = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled  = scaler_y.transform(y_test)

In [386]:
pca=PCA(n_components=0.95)
X_train_pca=pca.fit_transform(X_train_scaled)
X_test_pca=pca.transform(X_test_scaled)

In [387]:
ann_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_pca.shape[1],)),
    BatchNormalization(), Dropout(0.2),
    Dense(64, activation='relu'), BatchNormalization(), Dropout(0.2),
    Dense(32, activation='relu'), Dropout(0.1),
    Dense(16, activation='relu'),
    Dense(7, activation='linear')
])
ann_model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_51 (Dense)                │ (None, 128)            │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,391 (48.40 KB)

 Trainable params: 12,007 (46.90 KB)

 Non-trainable params: 384 (1.50 KB)

In [388]:
#Compile the model
ann_model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse', metrics=['mae'])

In [389]:
#Setup Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [390]:
#Setup Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

In [391]:
print("Training Optimized ANN...")
history = ann_model.fit(X_train_pca, y_train_scaled, validation_data=(X_test_pca, y_test_scaled), epochs=300, batch_size=16,
                  callbacks=[early_stopping, tensorboard_callback])

Training Optimized ANN...
Epoch 1/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 173ms/step - loss: 1.3099 - mae: 0.8237 - val_loss: 0.8240 - val_mae: 0.6505
Epoch 2/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 1.4784 - mae: 0.8410 - val_loss: 0.7821 - val_mae: 0.6319
Epoch 3/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 1.2171 - mae: 0.7541 - val_loss: 0.7551 - val_mae: 0.6204
Epoch 4/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 1.0854 - mae: 0.7444 - val_loss: 0.7356 - val_mae: 0.6123
Epoch 5/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 1.1579 - mae: 0.7417 - val_loss: 0.7270 - val_mae: 0.6090
Epoch 6/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 1.0151 - mae: 0.7068 - val_loss: 0.7234 - val_mae: 0.6076
Epoch 7/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 1.0265 - mae: 0.6900 - val_loss: 0.7126 - val_mae: 0.6027
Epoch 8/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 1.0226 - mae: 0.6915 - val_loss: 0.7010 - val_mae: 0.5975
Epoch 9/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/

In [392]:
print("Predicting on Test Set...")
y_pred_scaled = ann_model.predict(X_test_pca)

#Convert back to real percentages
y_pred_real = scaler_y.inverse_transform(y_pred_scaled)
y_test_real = scaler_y.inverse_transform(y_test_scaled)

#Calculate Metrics on the Real Data
r2 = r2_score(y_test_real, y_pred_real, multioutput='uniform_average')
mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))

print("ANN PERFORMANCE REPORT")
print(f"R2 Score: {r2:.4f}")
print(f"MAE:      {mae:.4f} %")
print(f"RMSE:     {rmse:.4f} %")
print("-" * 100)

results_df = pd.DataFrame(
    data=np.hstack((y_test_real, y_pred_real)),
    columns=['True_CetaneNumber', 'True_AnilinePoint', 'True_FreezePoint', 'True_PourPoint', 'True_CloudPoint', 'True_TotalAcidNumber', 'True_CtoHRatioByWt',
             'Pred_CetaneNumber', 'Pred_AnilinePoint', 'Pred_FreezePoint', 'Pred_PourPoint', 'Pred_CloudPoint', 'Pred_TotalAcidNumber', 'Pred_CtoHRatioByWt']   
)
print("\nSample Predictions (First 5 Rows):")
display(results_df.head())

Predicting on Test Set...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
ANN PERFORMANCE REPORT
R2 Score: 0.2376
MAE:      11.2762 %
RMSE:     21.1825 %
----------------------------------------------------------------------------------------------------

Sample Predictions (First 5 Rows):


,True_CetaneNumber,True_AnilinePoint,True_FreezePoint,True_PourPoint,True_CloudPoint,True_TotalAcidNumber,True_CtoHRatioByWt,Pred_CetaneNumber,Pred_AnilinePoint,Pred_FreezePoint,Pred_PourPoint,Pred_CloudPoint,Pred_TotalAcidNumber,Pred_CtoHRatioByWt
0,43.710569,91.597817,22.931187,82.545146,87.444495,0.882494,13.082205,46.318958,81.019089,46.191010,22.451046,47.305992,1.287615,9.747390
1,57.443852,76.496875,1.073170,13.930097,40.746252,1.127930,7.499064,49.319675,78.264290,35.728561,16.671524,54.055180,1.258840,9.815867
2,51.691607,76.332851,-3.970990,6.729892,33.794148,0.137409,7.752282,47.010448,55.846367,-3.844577,9.053753,33.815857,0.378277,7.571247
3,51.339091,80.330101,161.779073,33.819187,38.039479,0.979552,7.579958,51.303848,78.768784,41.340256,35.005333,55.336143,0.745449,8.608650
4,54.612271,73.151121,-12.328133,11.761402,39.177499,0.265074,7.181353,49.096249,54.308487,-8.191134,3.943223,32.414276,0.361994,7.544991
